In [32]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [33]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [34]:
with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [35]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [36]:
vocab = pd.read_csv('../static/model/vocabulary.txt', header=None)
tokens = vocab[0].tolist()

In [37]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [38]:
def preprocessing(text):
    data = pd.DataFrame([text], columns=['tweet'])
    data["Message"] = data["Message"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data["Message"] = data['Message'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data["Message"] = data["Message"].apply(remove_punctuations)
    # data["Message"] = data["Message"].str.replace('\d+', '', regex=True)
    data["Message"] = data["Message"].str.replace(r'\d+', '', regex=True)
    data["Message"] = data["Message"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data["Message"] = data["Message"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data["Message"]

In [39]:
def vectorizer(ds, vocabulary):
    vectorized_lst = []
    
    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))
        
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1
                
        vectorized_lst.append(sentence_lst)
        
    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)
    
    return vectorized_lst_new

In [40]:
def get_prediction(vectorized_text):
    prediction = model.predict(vectorized_text)
    if prediction == 1:
        return 'Negative'
    else:
        return 'Positive'

In [41]:
txt = "Malisha Rasiru"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

KeyError: 'Message'